<a href="https://www.kaggle.com/code/farrelad/dss-electre?scriptVersionId=233638364" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# PREPARATION

## Import Required Library

In [1]:
import math
from itertools import permutations
from itertools import product
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import json

## Global Helper Function

In [2]:
def create_criteria_decision_form() -> None: 
    global total_criteria
    
    dropdowns = []
    for i in range(total_criteria):
        label = widgets.Label(value=f"C{i+1}", layout=widgets.Layout(width="50px"))
    
        dropdown = widgets.Dropdown(options=['Benefit', 'Cost'], layout=widgets.Layout(width="100px"))
        dropdowns.append(dropdown)
    
        display(widgets.HBox([label, dropdown]))

    def get_dropdown_values():
        return [dropdown.value for dropdown in dropdowns]
    
    output = widgets.Output()
    
    def set_criteria(_):
        global criterias
        
        criterias['type'] = get_dropdown_values()
            
        with output:
            output.clear_output(wait=True)
            print('Criterias')
            criterias
        
    set_criteria_btn = widgets.Button(description="Set Criteria!")
    set_criteria_btn.on_click(set_criteria)

    display(set_criteria_btn, output)

In [3]:
def create_matrix_form() -> None:
    global total_criteria, total_alternative, criteria_labels, alternative_labels

    inputs = []
    for i in range(total_alternative):
        rows = []
        for c in range(total_criteria):
            rows.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
        inputs.append(rows)

    header_row = []
    for i in range(len(criteria_labels) + 1):
        header_row.append(widgets.Label(
            value='' if i == 0 else criteria_labels[i - 1],
            layout=widgets.Layout(width="100px")
        ))
    
    input_rows = []
    for r in range(total_alternative):
        row_widgets = [widgets.Label(value=alternative_labels[r], layout=widgets.Layout(width="50px"))] + inputs[r]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return np.array([[cell.value for cell in row] for row in inputs])

    output = widgets.Output()
    
    def update_matrix(_):
        global decision_matrix
        
        decision_matrix.loc[:, :] = get_matrix()

        with output:
            print("Decision matrix:")
            decision_matrix

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    display(widgets.HBox(header_row))
    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

In [4]:
def create_matrix_weight() -> None:
    global total_criteria, criteria_labels

    inputs = []
    for i in range(total_criteria):
        inputs.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
    
    input_rows = []
    for r in range(total_criteria):
        row_widgets = [widgets.Label(value=criteria_labels[r], layout=widgets.Layout(width="100px"))] + [inputs[r]]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return [row.value for row in inputs]

    output = widgets.Output()
    
    def update_matrix(_):
        global weights
        
        weights['weight'] = get_matrix()

        with output:
            print("Weight Criteria:")
            weights

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

---

# Input Starter Matrix-Like Data Structure of Criterias and Alternatives

In [5]:
total_criteria = 5
total_alternative = 3

criteria_labels = tuple(f"C{i + 1}" for i in range(total_criteria))
alternative_labels = tuple(f"A{i + 1}" for i in range(total_alternative))

Define criterias

In [6]:
criterias = pd.DataFrame(None, index=criteria_labels, columns=['type'])
criterias

,type
C1,NaN
C2,NaN
C3,NaN
C4,NaN
C5,NaN


Decide which criteria is benefit or cost

In [7]:
create_criteria_decision_form()

Button(description='Set Criteria!', style=ButtonStyle())

Output()

Create matrix for decision

In [8]:
decision_matrix = pd.DataFrame(0, index=alternative_labels, columns=criteria_labels)
decision_matrix

,C1,C2,C3,C4,C5
A1,0,0,0,0,0
A2,0,0,0,0,0
A3,0,0,0,0,0


In [9]:
create_matrix_form()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Normalization

Formula:

**Benefit criteria:**
$$
r_{ij} = \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

**Cost criteria:**
$$
r_{ij} = 1 - \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

In [10]:
matrix_r = decision_matrix.copy(deep=True).astype(float)

for i in range(total_criteria):
    sum_col = (decision_matrix.iloc[:, i] ** 2).sum()

    for j, row in enumerate(decision_matrix.itertuples(index=False)):
        if criterias.iloc[i]['type'] == 'Benefit':
            matrix_r.iloc[j, i] = decision_matrix.iloc[j, i] / math.sqrt(sum_col) # benefit criteria normalization
        else:
            matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization

matrix_r

<ipython-input-10-9505aab243af>:10: RuntimeWarning: invalid value encountered in divide
  matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization
<ipython-input-10-9505aab243af>:10: RuntimeWarning: invalid value encountered in divide
  matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization
<ipython-input-10-9505aab243af>:10: RuntimeWarning: invalid value encountered in divide
  matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization
<ipython-input-10-9505aab243af>:10: RuntimeWarning: invalid value encountered in divide
  matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization
<ipython-input-10-9505aab243af>:10: RuntimeWarning: invalid value encountered in divide
  matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization
/usr/local/lib/pytho

,C1,C2,C3,C4,C5
A1,NaN,NaN,NaN,NaN,NaN
A2,NaN,NaN,NaN,NaN,NaN
A3,NaN,NaN,NaN,NaN,NaN


# Define Weights

In [11]:
weights = pd.DataFrame(0, index=criteria_labels, columns=['weight'])
weights

,weight
C1,0
C2,0
C3,0
C4,0
C5,0


In [12]:
create_matrix_weight()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Calculate Weighted Matrix

Formula:

$$
v_{ij} = r_{ij} \cdot w_{ij}
$$

In [13]:
weighted_matrix = matrix_r.mul(weights['weight'], axis=1)
weighted_matrix

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,C1,C2,C3,C4,C5
A1,NaN,NaN,NaN,NaN,NaN
A2,NaN,NaN,NaN,NaN,NaN
A3,NaN,NaN,NaN,NaN,NaN


# Get The Concordance and Discordance Set

Calculate the permutation of alternative to see how many group should concordance and discordance have

In [14]:
permutation_alternative = math.perm(total_alternative, 2)
permutation_alternative

6

## Concordance

In [15]:
concordances_data = []
for a, b in permutations(alternative_labels, 2):
    row_1 = int(a[-1]) - 1
    row_2 = int(b[-1]) - 1

    selected_criteria = []
    for i in range(len(weighted_matrix.columns)):
        if weighted_matrix.iloc[row_1, i] >= weighted_matrix.iloc[row_2, i]:
            selected_criteria.append([a, b, f"C{i+1}"])

    is_a_found = any(row[0] == a for row in selected_criteria)
    is_b_found = any(row[1] == b for row in selected_criteria)

    if is_a_found and is_b_found:
        for i in selected_criteria:
            concordances_data.append(i)
    else:
        concordances_data.append([a, b, None])

concordances = pd.DataFrame(concordances_data, columns=['First', 'Second', 'Selected Criteria'])
concordances

,First,Second,Selected Criteria
0,A1,A2,None
1,A1,A3,None
2,A2,A1,None
3,A2,A3,None
4,A3,A1,None
5,A3,A2,None


## Discordance

In [16]:
discordances_data = []
for a, b in permutations(alternative_labels, 2):
    row_1 = int(a[-1]) - 1
    row_2 = int(b[-1]) - 1

    selected_criteria = []
    for i in range(len(weighted_matrix.columns)):
        if weighted_matrix.iloc[row_1, i] < weighted_matrix.iloc[row_2, i]:
            selected_criteria.append([a, b, f"C{i+1}"])

    is_a_found = any(row[0] == a for row in selected_criteria)
    is_b_found = any(row[1] == b for row in selected_criteria)

    if is_a_found and is_b_found:
        for i in selected_criteria:
            discordances_data.append(i)
    else:
        discordances_data.append([a, b, None])

discordances = pd.DataFrame(discordances_data, columns=['First', 'Second', 'Selected Criteria'])
discordances

,First,Second,Selected Criteria
0,A1,A2,None
1,A1,A3,None
2,A2,A1,None
3,A2,A3,None
4,A3,A1,None
5,A3,A2,None


# Define The Matrix of Concordance and Discordance

## Concordance

Formula

$$
c_{kl} = \Sigma_{j \in c_{kl}} W_j
$$

In [17]:
sum_weights_concordance = {f"{a[-1]}-{b[-1]}" : 0 if a != b else None for a, b in product(alternative_labels, repeat=2)}

for val in concordances.itertuples(index=False):
    sum_weights_concordance[val[0][-1]+'-'+val[1][-1]] += weights.loc[val[2]].weight if val[2] != None else 0

# print(json.dumps(sum_weights_concordance, indent=4))

Convert it to matrix-like data structure (Pandas DataFrame)

In [18]:
last_dict_item_concordance = list(sum_weights_concordance.keys())[-1]

concordance_matrix = pd.DataFrame(np.nan, index=range(int(last_dict_item_concordance[0])), columns=range(int(last_dict_item_concordance[-1])))

for key, value in sum_weights_concordance.items():
    row, col = map(int, key.split('-'))
    concordance_matrix.at[row-1, col-1] = value
    
concordance_matrix

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,0,1,2
0,NaN,0.0,0.0
1,0.0,NaN,0.0
2,0.0,0.0,NaN


## Discordance

Formula

$$
d_{kl} = \frac{max(|v_{kj} - v_{vj}|)_{j \in D_{kl}}}{max(|v_{kj} - v_{lj}|)_{\forall_{j}}}
$$

In [19]:
discordances_group = {f"{a[-1]}-{b[-1]}" : [] if a != b else None for a, b in product(alternative_labels, repeat=2)}
# print(json.dumps(discordances_group, indent=4))

In [20]:
for val in discordances.itertuples(index=False):
    a = val[0][-1]
    b = val[1][-1]
    discordances_group[a+'-'+b].append(val[2])

# print(json.dumps(discordances_group, indent=4))

In [21]:
discordance_matrix_data = {f"{a[-1]}-{b[-1]}" : 0 if a != b else None for a, b in product(alternative_labels, repeat=2)}

for key, val in discordances_group.items():
    if val == None: continue
        
    if tuple(val) not in weighted_matrix.columns: continue
        
    a1 = f"A{key[0]}"
    a2 = f"A{key[-1]}"

    max_difference_row = abs(weighted_matrix.loc[a1] - weighted_matrix.loc[a2]).max()
    max_difference_criteria_discordance = abs(weighted_matrix.loc[a1, val] - weighted_matrix.loc[a2, val]).max()

    discordance_matrix_data[key] = max_difference_criteria_discordance / max_difference_row

# print(json.dumps(discordance_matrix_data, indent=4))

Convert data structure to matrix-like data type (Pandas DataFrame)

In [22]:
last_dict_item_discordance = list(discordance_matrix_data.keys())[-1]

discordance_matrix = pd.DataFrame(np.nan, index=range(int(last_dict_item_discordance[0])), columns=range(int(last_dict_item_discordance[-1])))

for key, value in discordance_matrix_data.items():
    row, col = map(int, key.split('-'))
    discordance_matrix.at[row-1, col-1] = value
    
discordance_matrix

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,0,1,2
0,NaN,0.0,0.0
1,0.0,NaN,0.0
2,0.0,0.0,NaN


# Calculate the Dominant of The Matrix

Threshold formula:

$$
\underline{c} = \frac{\Sigma_{k=1}^m \Sigma_{l=1}^m c_{kl}}{m(m-1)}
$$

In [23]:
def calculate_threshold(matrix: pd.DataFrame):
    return matrix.sum().sum() / (matrix.shape[0] * (matrix.shape[0] - 1))

## Concordance

Compare each value in matrix with **threshold (c)**

So, the value of matrix F is:
$$
f_{kl} = 
\begin{cases}
    1, \text{if } c_{kl} \geq \underline{c} \\
    0, \text{if } c_{kl} \lt \underline{c}
\end{cases}
$$

In [24]:
concordance_threshold = calculate_threshold(concordance_matrix)
concordance_threshold

0.0

In [25]:
dominant_concordance = concordance_matrix.applymap(lambda x: 1 if pd.notna(x) and x > concordance_threshold else (0 if pd.notna(x) else x))
dominant_concordance

<ipython-input-25-f1d6bdb027bb>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dominant_concordance = concordance_matrix.applymap(lambda x: 1 if pd.notna(x) and x > concordance_threshold else (0 if pd.notna(x) else x))
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,0,1,2
0,NaN,0.0,0.0
1,0.0,NaN,0.0
2,0.0,0.0,NaN


## Discordance

Compare each value in matrix with **threshold (c)**.

So, the value of matrix F is:
$$
g_{kl} = 
\begin{cases}
    1, \text{if } d_{kl} \geq \underline{d} \\
    0, \text{if } d_{kl} \lt \underline{d}
\end{cases}
$$

In [26]:
discordance_threshold = calculate_threshold(discordance_matrix)
discordance_threshold

0.0

In [27]:
dominant_discordance = discordance_matrix.applymap(lambda x: 1 if pd.notna(x) and x > discordance_threshold else (0 if pd.notna(x) else x))
dominant_discordance

<ipython-input-27-aaa0fcdabd1d>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dominant_discordance = discordance_matrix.applymap(lambda x: 1 if pd.notna(x) and x > discordance_threshold else (0 if pd.notna(x) else x))
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,0,1,2
0,NaN,0.0,0.0
1,0.0,NaN,0.0
2,0.0,0.0,NaN


# Calculate Aggregate Dominant Matrix

Formula:

$$
e_{kl} = f_{kl} \times g_{kl}
$$
or
$$
E = F \odot G
$$

In [28]:
aggregate_dominant_matrix = dominant_concordance * dominant_discordance
aggregate_dominant_matrix

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,0,1,2
0,NaN,0.0,0.0
1,0.0,NaN,0.0
2,0.0,0.0,NaN


# Eliminate Less Favourable Alternatives

In [29]:
selected_alternatives = aggregate_dominant_matrix[aggregate_dominant_matrix.eq(1).any(axis=1)]
print(selected_alternatives)

if selected_alternatives.empty:
    print('\nYou should evaluate and rank the alternatives before deciding!')
else:
    potential_alternatives = selected_alternatives[selected_alternatives.eq(1).any(axis=1)]
    potential_alternatives

Empty DataFrame
Columns: [0, 1, 2]
Index: []

You should evaluate and rank the alternatives before deciding!


# Ranking Alternative

In [30]:
rank_table = concordance_matrix - discordance_matrix

rank_table['total_point'] = rank_table.sum(axis=1)

rank_table = rank_table.sort_values(by='total_point', ascending=False)
rank_table['rank'] = range(1, len(rank_table) + 1)

rank_table['alternative_name'] = rank_table['rank'].apply(lambda x: f"A{x+1}")

rank_table

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,0,1,2,total_point,rank,alternative_name
0,NaN,0.0,0.0,0.0,1,A2
1,0.0,NaN,0.0,0.0,2,A3
2,0.0,0.0,NaN,0.0,3,A4
